# Fit `polyclonal` model for antibody.

Import Python modules.

In [ ]:
import altair as alt

import polyclonal

import pandas as pd

This notebook is parameterized by `papermill`.
The next cell is tagged as `parameters` to get the passed parameters.

In [ ]:
# this cell is tagged parameters for `papermill` parameterization
selection = None
params = None